<a href="https://colab.research.google.com/github/RainingCodes/Simple_Transformer/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_text as text  # 한국어 처리를 위해 필요
import numpy as np

In [ ]:
# 1. Tokenizer 정의 (Word-based에서 Vectorization 사용 / Tokinizer : 단순히 "문장을 숫자로 변환"하는 것이 아니라, 적절한 방식으로 나누고(토큰화) 변환하는 과정)
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=10000, output_mode="int", output_sequence_length=5
)


In [ ]:
# 2. 학습 데이터 (한국어 → 영어 번역 예제)
train_sentences = [
    "나는 학생입니다",  # Input (한국어)
    "그녀는 선생님입니다",
    "오늘 날씨가 좋네요"
]
target_sentences = [
    "I am a student",  # Output (영어)
    "She is a teacher",
    "Today's weather is good"
]

In [1]:
# Tokenizer 학습 (텍스트를 숫자로 변환할 수 있도록 어휘 사전 구축)
vectorizer.adapt(train_sentences)

In [ ]:
# 3. Self-Attention 블록 정의
class SimpleSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim):
        super(SimpleSelfAttention, self).__init__()
        self.W_q = tf.keras.layers.Dense(dim)  # Query 변환 행렬
        self.W_k = tf.keras.layers.Dense(dim)  # Key 변환 행렬
        self.W_v = tf.keras.layers.Dense(dim)  # Value 변환 행렬

    def call(self, x):
        Q = self.W_q(x)  # Query 행렬 생성
        K = self.W_k(x)  # Key 행렬 생성
        V = self.W_v(x)  # Value 행렬 생성

        # Self-Attention의 핵심 연산: 가중치 계산
        attention_scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(K.shape[-1], tf.float32))
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)

        # 중요도가 높은 단어일수록 더 반영됨
        return tf.matmul(attention_weights, V)


In [ ]:
# 4. Transformer 모델 정의 (Embedding + Self-Attention)
class SimpleTransformer(tf.keras.Model):
    def __init__(self, vocab_size, dim=16):
        super(SimpleTransformer, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, dim)  # 단어를 의미 공간에 매핑
        self.self_attention = SimpleSelfAttention(dim)  # Self-Attention 적용
        self.dense = tf.keras.layers.Dense(10, activation="softmax")  # 최종 출력

    def call(self, x):
        x = self.embedding(x)  # 문장을 벡터 공간으로 변환
        x = self.self_attention(x)  # Self-Attention 적용하여 관계 파악
        return self.dense(x)  # 변환된 결과

In [ ]:
# 5. 모델 생성 및 학습
vocab_size = 10000  # 어휘 사전 크기
model = SimpleTransformer(vocab_size)


In [ ]:
# 입력 데이터 준비 (문장을 벡터로 변환)
X_train = vectorizer(train_sentences)
Y_train = vectorizer(target_sentences)


In [ ]:
# 모델 학습
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(X_train, Y_train, epochs=10)

In [ ]:
# 6. 추론 (Inference) 테스트
test_sentence = "나는 학생입니다"
test_vector = vectorizer([test_sentence])  # 변환된 숫자 입력
output = model(test_vector)

In [ ]:
# 7. 결과 해석
print("입력 문장:", test_sentence)
print("출력 벡터:", output.numpy())  # 예측된 벡터 출력